# Scientific Computing in Python: A NumPy Crash-Course Cluedump

Mark Chilenski (markchil)

January 25, 2022

You can download this notebook and supporting materials at https://github.com/markchil/numpy-lecture

Copyright 2021 Mark Chilenski

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.

## Introduction
NumPy is the main Python package for performing numerical computations

* NumPy provides efficient data structures and functions for manipulating arrays of numbers
* You can lots of computations with NumPy itself, or can hand off NumPy arrays for use in other specialized packages such as SciPy

NumPy is usually imported as `np`:

In [70]:
import numpy as np

## Example: Evaluating a 2d Quadratic
Let's evaluate $z = ax^2 + by^2 + cxy$ over a 2d grid covering $-1\leq x, y\leq 1$. First, we define the coefficients as regular Python floats:

In [2]:
a = 5.0
b = -2.0
c = 3.0

### Pure Python Solution:

A pure Python solution requires some ugly syntax to set up the grids for $x$ and $y$:

In [3]:
x = [2.0 * v / (200 - 1) - 1.0 for v in range(200)]
y = [2.0 * v / (201 - 1) - 1.0 for v in range(201)]

Evaluating $z$ requires nested for-loops:

In [4]:
def quadratic_pure_python(x, y):
    z = []
    for x_val in x:
        row = []
        for y_val in y:
            row.append(a * x_val ** 2 + b * y_val ** 2 + c * x_val * y_val)
        z.append(row)
    return z

In [5]:
%timeit quadratic_pure_python(x, y)

13.7 ms ± 993 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### NumPy Solution:

The NumPy `linspace` function provides a nice way of making a grid of evenly-spaced points:

In [6]:
x = np.linspace(-1.0, 1.0, 200)
y = np.linspace(-1.0, 1.0, 201)

We can evaluate $z$ over the entire $210\times190$ point grid without a loop:

In [7]:
def quadratic_numpy():
    z = (
        a * x[:, np.newaxis] ** 2 + b * y[np.newaxis, :] ** 2 +
        c * x[:, np.newaxis] * y[np.newaxis, :]
    )
    return z

In [8]:
%timeit quadratic_numpy()

137 µs ± 15.7 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


NumPy runs $100\times$ faster, and has cleaner syntax! We get this speedup because NumPy provides a bridge from the slow (but flexible) world of Python to fast compiled C/C++/Fortran code.

## Storing $N$-Dimensional Data: The `ndarray` Class
* The `ndarray` class (usually just called "array" or "numpy array") stores arrays of $N$-dimensional data
* A given array `x` consists of:
    * A chunk of memory
    * A datatype which indicates what type of value is stored in the memory (`x.dtype`)
    * A shape which indicates how the elements are arranged in an $N$-dimensional grid (`x.shape`)

## Creating Arrays

You can create an array from an existing list:

In [9]:
x = np.asarray([1, 2, 3, 4], dtype=float)
print(x)

[1. 2. 3. 4.]


(Note: NumPy can try to guess `dtype` based on the input, but it is usually best to be explicit: if we didn't specify `dtype=float` here, it would have defaulted to `int`.)

You can create multidimensional arrays using nested lists:

In [10]:
x = np.asarray([[1, 2, 3], [4, 5, 6]], dtype=float)
print(x)

[[1. 2. 3.]
 [4. 5. 6.]]


There is a special function to get an array of zeros:

In [11]:
x = np.zeros((3, 2))
print(x)

[[0. 0.]
 [0. 0.]
 [0. 0.]]


(Note that the shape is indicated as a tuple of ints.)

An array of ones:

In [12]:
x = np.ones(5)
print(x)

[1. 1. 1. 1. 1.]


(Note that a 1d array can be obtained by passing a single int for the shape, rather than a tuple.)

If you have an existing array, you can easily create a new array with the same size and dtype:

In [13]:
x = np.zeros((3, 2), dtype=int)
y = np.ones_like(x)
z = np.zeros_like(x)
print(x)
print(y)
print(z)

[[0 0]
 [0 0]
 [0 0]]
[[1 1]
 [1 1]
 [1 1]]
[[0 0]
 [0 0]
 [0 0]]


You can also override the dtype, if necessary:

In [14]:
x = np.zeros((3, 2), dtype=int)
y = np.ones_like(x, dtype=float)
print(x)
print(y)

[[0 0]
 [0 0]
 [0 0]]
[[1. 1.]
 [1. 1.]
 [1. 1.]]


Linearly-spaced numbers:

In [15]:
x = np.linspace(0, 1, 4)
print(x)

[0.         0.33333333 0.66666667 1.        ]


Logarithmically-spaced numbers:

In [16]:
x = np.logspace(-1, 1, 5)
print(x)

[ 0.1         0.31622777  1.          3.16227766 10.        ]


An identity matrix:

In [17]:
x = np.eye(5)
print(x)

[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]


## Getting Elements Into (and out of) Arrays
### 1d Arrays
You can access single elements using square brackets, just like with Python lists:

In [18]:
x = np.linspace(0, 1, 5)
print(x)

[0.   0.25 0.5  0.75 1.  ]


In [19]:
print(x[0])

0.0


You can use negative indices:

In [20]:
print(x[-1])

1.0


You can access a range of elements using slice notation:

In [21]:
print(x[1:3])

[0.25 0.5 ]


(Note: just like with Python lists, the slice is a half-open interval: `x[1:3]` gets the elements at indices 1 through 2.)

You can specify a stride for the slice:

In [22]:
print(x[::2])

[0.  0.5 1. ]


(Note: `x[::2]` is a shorthand for `x[0:len(x):2]`. Slices in genreal have the form `start:stop:stride`. Matlab/Octave users should note that the stride is the **last** number in a Python slice.)

You can use an array (or list) of integers to extract specific indices:

In [23]:
print(x[[0, 2, 3]])

[0.   0.5  0.75]


(Note: the double square brackets "`[[`" are necessary!)

You can use an array (or list) of booleans with the same length as the array to extract specified elements:

In [24]:
print(x[[True, False, True, True, False]])

[0.   0.5  0.75]


The use of boolean expressions for indexing is extremely powerful:

In [25]:
print(x[x >= 0.5])

[0.5  0.75 1.  ]


### 2d Arrays
You can provide a comma-separated list of any of the types of indexing shown above to extract elements from an 2-dimensional array:

In [26]:
x = np.random.randn(3, 4)
print(x)

[[-2.45249503  1.39642535 -0.1116106  -0.86880824]
 [ 0.08387987 -0.44749853 -0.84368502  0.03063071]
 [ 0.58542764  1.06357497  0.69214984 -0.67683935]]


The first index is the row index, the second index is the column index:

In [27]:
print(x[0, 1])

1.3964253519782508


To get all of the entries along a given axis (e.g., an entire row), use the slice operator `:` with no start, end, or stride indicated:

In [28]:
print(x[0, :])

[-2.45249503  1.39642535 -0.1116106  -0.86880824]


You can omit the trailing `:` when accessing entire rows:

In [29]:
print(x[0])

[-2.45249503  1.39642535 -0.1116106  -0.86880824]


Indexing with boolean arrays is a little more complicated. You can use a mask which has the same shape as `x` to get a 1d array of all elements for which the mask is `True`:

In [30]:
print(x[x > 0])

[1.39642535 0.08387987 0.03063071 0.58542764 1.06357497 0.69214984]


Or you can provide a mask for only one dimension:

In [31]:
print(x[[True, False, True], :])

[[-2.45249503  1.39642535 -0.1116106  -0.86880824]
 [ 0.58542764  1.06357497  0.69214984 -0.67683935]]


### Higher-Dimensional Arrays
Pretty much the same as 2-dimensional: provide a comma-separated list of indices, one for each dimension.

In [32]:
x = np.random.randn(3, 4, 5)

You can omit any number of trailing `:`'s:

In [33]:
print(x[0, :, :])
print(x[0])

[[-0.12781729  0.99219555  0.88809274  0.08918892 -0.08443868]
 [ 0.48477381 -0.866477    0.4513086  -0.31376352  0.31122344]
 [-0.10703685  1.83829395  0.56713457  0.16880293 -2.2665751 ]
 [ 0.64324272 -1.19160025  1.22910789  1.70621644 -0.18970577]]
[[-0.12781729  0.99219555  0.88809274  0.08918892 -0.08443868]
 [ 0.48477381 -0.866477    0.4513086  -0.31376352  0.31122344]
 [-0.10703685  1.83829395  0.56713457  0.16880293 -2.2665751 ]
 [ 0.64324272 -1.19160025  1.22910789  1.70621644 -0.18970577]]


You can omit any number of leading `:`'s by using `...`:

In [34]:
print(x[:, :, 0])
print(x[..., 0])

[[-0.12781729  0.48477381 -0.10703685  0.64324272]
 [-0.69949167  0.33128595  1.70746623  0.92048578]
 [ 0.36946969 -0.36270725 -1.42711581 -0.13028131]]
[[-0.12781729  0.48477381 -0.10703685  0.64324272]
 [-0.69949167  0.33128595  1.70746623  0.92048578]
 [ 0.36946969 -0.36270725 -1.42711581 -0.13028131]]


### Adding New Dimensions With Indexing: `np.newaxis`
You can add a new dimension (with length 1) to an array by including `np.newaxis` in the list of index expressions. This will be very useful once we introduce the concept of broadcasting, below.

In [35]:
x = np.random.randn(4, 3)
y = x[:, np.newaxis, :]
print(x.shape)
print(y.shape)

(4, 3)
(4, 1, 3)


## Array Operations
The key to NumPy's power is the ability to do fast *vectorized* operations. "Vectorized" simply means that you can perform the same operation on every element of an array with a single function call -- no need to write a loop. In fact, you should avoid writing loops whenever possible, as when performing an operation on all of the elements of an array it is almost always faster (and cleaner) to do a single call to a NumPy function.

### Unary Operations on Arrays
Unary operators act element-by-element on arrays. This makes the syntax for performing calculations over a grid of inputs very clean, and is also much faster than using loops.

Let's test the speed difference using exponentiation:

In [36]:
x = np.linspace(0, 1, 5000)

In [37]:
def square_loop(x):
    y = np.zeros_like(x)
    for idx, x_val in enumerate(x):
        y[idx] = x_val ** 2
    return y

In [38]:
%timeit square_loop(x)

2.92 ms ± 428 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [39]:
def square_numpy(x):
    return x ** 2

In [40]:
%timeit square_numpy(x)

6.98 µs ± 1.44 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Using the vectorized version is over 500x faster!

To illustrate some of the main unary operations, let's use a smaller array:

In [41]:
x = np.linspace(0, 1, 5)
print(x)

[0.   0.25 0.5  0.75 1.  ]


Addition of a scalar:

In [42]:
y = x + 10
print(y)

[10.   10.25 10.5  10.75 11.  ]


Multiplication by a scalar:

In [43]:
y = 5 * x
print(y)

[0.   1.25 2.5  3.75 5.  ]


Division by a scalar:

In [44]:
y = x / 2
print(y)

[0.    0.125 0.25  0.375 0.5  ]


Negation:

In [45]:
y = -x
print(y)

[-0.   -0.25 -0.5  -0.75 -1.  ]


Exponentiation:

In [46]:
y = x ** 2
print(y)

[0.     0.0625 0.25   0.5625 1.    ]


Square root:

In [47]:
y = np.sqrt(x)
print(y)

[0.         0.5        0.70710678 0.8660254  1.        ]


Exponential ($y=e^x$):

In [48]:
y = np.exp(x)
print(y)

[1.         1.28402542 1.64872127 2.11700002 2.71828183]


(Natural) Logarithm:

In [49]:
y = np.log(x)
print(y)

[       -inf -1.38629436 -0.69314718 -0.28768207  0.        ]


/var/folders/1d/t26w870175q9__c1qp_d1w4r0000gp/T/ipykernel_41345/2719270062.py:1: RuntimeWarning: divide by zero encountered in log
  y = np.log(x)


(Note: $\log(0)$ evaluates to the floating point value `-inf` and raises a warning.)

Trigonometric sine:

In [50]:
y = np.sin(x)
print(y)

[0.         0.24740396 0.47942554 0.68163876 0.84147098]


### Binary Operations on Equally-Sized Arrays
Binary operations also act element-by-element. Let's illustrate that with the following arrays:

In [51]:
x = np.linspace(0, 1, 5)
y = np.linspace(1, 2, 5)
print(x)
print(y)

[0.   0.25 0.5  0.75 1.  ]
[1.   1.25 1.5  1.75 2.  ]


Addition:

In [52]:
z = x + y
print(z)

[1.  1.5 2.  2.5 3. ]


Multiplication:

In [53]:
z = x * y
print(z)

[0.     0.3125 0.75   1.3125 2.    ]


Division:

In [54]:
z = y / x
print(z)

[       inf 5.         3.         2.33333333 2.        ]


/var/folders/1d/t26w870175q9__c1qp_d1w4r0000gp/T/ipykernel_41345/793868733.py:1: RuntimeWarning: divide by zero encountered in true_divide
  z = y / x


(Note: division by zero returns the floating point value `inf` and raises a warning.)

Exponentiation:

In [55]:
z = x ** y
print(z)

[0.         0.1767767  0.35355339 0.60444559 1.        ]


### Broadcasting: Binary Operations on Differently-Sized Arrays
#### Basic Example
Broadcasting is a powerful syntax for efficiently performing calculations on "tensor product" grids of inputs. It allows you to write very fast, expressive code without needing loops -- we used this in the 2d quadratric example at the beginning. Let's take a closer look. Say we want to evaluate the cross-term `z=xy` on the grid $-1\leq x\leq 0$, $0\leq y\leq 1$:

In [71]:
x = np.linspace(-1, 0, 5)
y = np.linspace(0, 1, 6)

`x` and `y` have different shapes, so trying to call `x*y` raises an error:

In [76]:
# We use a try-catch so that the notebook can run through:
try:
    z = x * y
except ValueError as e:
    import traceback
    print(traceback.format_exc())

Traceback (most recent call last):
  File "/var/folders/1d/t26w870175q9__c1qp_d1w4r0000gp/T/ipykernel_41345/1434134433.py", line 2, in <module>
    z = x * y
ValueError: operands could not be broadcast together with shapes (5,) (6,) 



Instead, we can use `np.newaxis` to reshape `x` and `y`:

In [58]:
x_reshaped = x[:, np.newaxis]
y_reshaped = y[np.newaxis, :]
print(x_reshaped.shape)
print(y_reshaped.shape)

(5, 1)
(1, 6)


We can now compute the desired cross-term:

In [59]:
z = x_reshaped * y_reshaped
print(z.shape)

(5, 6)


Essentially what happens is NumPy acts as if `x_reshaped` has shape `(5, 6)`, where every column `x_reshaped[:, i]` is the same. But, it does this in a very memory-efficient way: you only ever have to store the 5 elements of `x_reshaped`, not the 30 elements of the expanded array. Similarly, NumPy acts as if `y_reshaped` has shape `(5, 6)`, where every row `y_reshaped[i, :]` is the same.

#### Full Details
Two arrays can be broadcast together if their shapes are compatible. To determine compatibility, use the following steps:
1. If one array has fewer dimensions than the other, add 1's at the *beginning* of the lower-dimensional array's shape
2. For each element of the shapes, either:
    * The two arrays must have the same number of elements along that dimension, or
    * One (or both) of the arrays must have 1 element along that dimension

Example:

In [60]:
x = np.zeros((2, 1, 3))
y = np.zeros((5, 1, 4, 3))
z = x + y
print('  ', x.shape)
print(y.shape)
print(z.shape)

   (2, 1, 3)
(5, 1, 4, 3)
(5, 2, 4, 3)


### Aggregation Operations on Arrays
The unary and binary operations we've looked at so far leave the array shapes unchanged (or expanded according to the broadcasting rules). There are also operations to compute aggregated quantities, which cause one or more dimensions to be dropped.

In [61]:
x = np.linspace(0, 1, 10)

Many of these are available as methods of the array class itself, including:

Sum of elements:

In [62]:
print(x.sum())

5.0


Minimum value:

In [63]:
print(x.min())

0.0


Maximum value:

In [64]:
print(x.max())

1.0


Mean value:

In [65]:
print(x.mean())

0.5


Standard deviation:

In [66]:
print(x.std())

0.31914236925211265


On multi-dimensional arrays, the default is to compute these operations across all elements in the array, so the result is a scalar:

In [67]:
x = np.ones((3, 4, 5, 6))
print(x.sum())

360.0


You can also specify an axis to apply the operation along:

In [68]:
s = x.sum(axis=3)
print(x.shape)
print(s.shape)

(3, 4, 5, 6)
(3, 4, 5)


Note that the output has one fewer dimension. Mathematically, what we now have is $s_{abc} = \sum_d x_{abcd}$.

We can also provide multiple dimensions to sum over:

In [69]:
s = x.sum(axis=(1, 3))
print(x.shape)
print(s.shape)

(3, 4, 5, 6)
(3, 5)


What we have in this case is $s_{ac}=\sum_b\sum_d x_{abcd}$

Sometimes we want to keep the dimension we aggregated over. For instance, suppose we want to normalize each row of a matrix by its 2-norm (using `np.linalg.norm`, a function which takes a single array as its argument). The shapes are not compatible, and hence trying to compute `x / x_sum` would raise an exception:

In [87]:
x = 10 * np.random.randn(3, 4)
x_norm = np.linalg.norm(x, axis=1)
print(x.shape)
print(x_norm.shape)

(3, 4)
(3,)


Instead, we can use the `keepdims` flag:

In [88]:
x_norm = np.linalg.norm(x, axis=1, keepdims=True)
x_normalized = x / x_norm
print(x.shape)
print(x_norm.shape)
print(x)
print(x_normalized)

(3, 4)
(3, 1)
[[-20.09077111  -8.18802668   1.10836107  -7.76810964]
 [  1.26406622  21.27165382  -6.83381774   3.2739478 ]
 [-19.83606315  -7.44545159 -16.69740254   1.18187441]]
[[-0.87083683 -0.35491098  0.04804204 -0.33670962]
 [ 0.05589161  0.94054168 -0.30216223  0.14475999]
 [-0.73461649 -0.27573775 -0.61837811  0.04377   ]]


## Doing Stuff With Arrays: The Scientific Python Ecosystem
Many packages use NumPy to move data around and accomplish various tasks. The ones I use the most are:
* NumPy (https://numpy.org/) itself provides subpackages for:
    * Linear algebra (`numpy.linalg`)
    * Taking discrete Fourier transforms (`numpy.fft`), and
    * Generating random numbers (`numpy.random`)
* SciPy (https://scipy.org/) was developed jointly with NumPy, and provides higher-level routines for:
    * Special functions (`scipy.special`)
    * Numeric integration (`scipy.integrate`)
    * Optimization (`scipy.optimize`)
    * Interpolation (`scipy.interpolate`)
    * More powerful Fourier transform capabilities (`scipy.fft`)
    * Signal processing (`scipy.signal`)
    * More powerful linear algebra capabilities (`scipy.linalg`)
    * Manipulating spatial data (`scipy.spatial`)
    * Statistics (`scipy.stats`)
    * Multidimensional image processing (`scipy.ndimage`)
    * Reading various file formats (`scipy.io`)
* Matplotlib (https://matplotlib.org/) provides a variety of plotting routines
* scikit-learn (https://scikit-learn.org/stable/) provides a **huge** range of machine learning algorithms and supporting tools